In [1]:
import xarray as xr
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings

In [2]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Relative Humidity/*.nc'
data=xr.open_mfdataset(path)


In [3]:
data

<xarray.Dataset>
Dimensions:    (level: 8, lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 925.0 850.0 700.0 600.0 500.0 400.0 300.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    rhum       (time, level, lat, lon) float32 dask.array<chunksize=(365, 8, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [4]:
data.level

<xarray.DataArray 'level' (level: 8)>
array([1000.,  925.,  850.,  700.,  600.,  500.,  400.,  300.], dtype=float32)
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 600.0 500.0 400.0 300.0
Attributes:
    units:         millibar
    actual_range:  [1000.  300.]
    long_name:     Level
    positive:      down
    GRIB_id:       100
    GRIB_name:     hPa
    axis:          Z

Relative Humidity : At levels: 1000,850,500,300 

# Extracting data

In [15]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
desired_levels=[1000,850,500,300]
# desired_levels=[300]

# Use the sel method to select data within the specified Bangladesh region
bangladesh_rhum = data.sel(lat=lat_range,lon=lon_range,level=desired_levels)

In [16]:
bangladesh_rhum

<xarray.Dataset>
Dimensions:    (level: 4, lat: 4, lon: 5, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 850.0 500.0 300.0
  * lat        (lat) float32 27.5 25.0 22.5 20.0
  * lon        (lon) float32 85.0 87.5 90.0 92.5 95.0
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    rhum       (time, level, lat, lon) float32 dask.array<chunksize=(365, 4, 4, 5), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [17]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
rhum_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'rhum_1000hpa','rhum_850hpa','rhum_500hpa','rhum_300hpa'])

# Calculate the number of chunks
num_chunks = len(bangladesh_rhum.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_rhum.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    rhum_1000hpa = chunk.sel(level=1000).rhum.values
    rhum_850hpa = chunk.sel(level=850).rhum.values
    rhum_500hpa = chunk.sel(level=500).rhum.values
    rhum_300hpa = chunk.sel(level=300).rhum.values
    

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'rhum_1000hpa': rhum_1000hpa.ravel(),
        'rhum_850hpa': rhum_850hpa.ravel(),
        'rhum_500hpa': rhum_500hpa.ravel(),
        'rhum_300hpa': rhum_300hpa.ravel()
    })

    rhum_df = rhum_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
rhum_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,rhum_1000hpa,rhum_850hpa,rhum_500hpa,rhum_300hpa
0,1981-01-01,27.5,85.0,54.750008,54.750008,11.25,17.50
1,1981-01-01,25.0,87.5,60.750008,60.750008,5.00,8.50
2,1981-01-01,22.5,90.0,72.250015,72.250015,9.25,3.50
3,1981-01-01,20.0,92.5,87.000015,87.000015,22.25,2.25
4,1981-01-01,27.5,95.0,99.250015,99.750015,25.50,4.50
...,...,...,...,...,...,...,...
306795,2022-12-31,20.0,85.0,58.250000,49.750000,13.25,12.00
306796,2022-12-31,27.5,87.5,62.750000,37.000000,12.50,12.75
306797,2022-12-31,25.0,90.0,57.000000,26.000000,11.00,13.75
306798,2022-12-31,22.5,92.5,54.250000,24.500000,11.75,13.00


In [18]:
rhum_df.head(60)

,time,lat,lon,rhum_1000hpa,rhum_850hpa,rhum_500hpa,rhum_300hpa
0,1981-01-01,27.5,85.0,54.750008,54.750008,11.250000,17.500000
1,1981-01-01,25.0,87.5,60.750008,60.750008,5.000000,8.500000
2,1981-01-01,22.5,90.0,72.250015,72.250015,9.250000,3.500000
3,1981-01-01,20.0,92.5,87.000015,87.000015,22.250000,2.250000
4,1981-01-01,27.5,95.0,99.250015,99.750015,25.500000,4.500000
5,1981-01-01,25.0,85.0,45.500008,23.000000,3.500000,12.250000
6,1981-01-01,22.5,87.5,61.750015,44.250000,4.000000,12.000000
7,1981-01-01,20.0,90.0,76.500015,74.000015,6.500000,11.250000
8,1981-01-01,27.5,92.5,78.250015,77.500015,5.250000,9.250000
9,1981-01-01,25.0,95.0,70.000015,61.500015,7.250000,7.000000
